In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("20190410-Roche-LN-pt_table.csv", sep='\t')

In [3]:
df.head()

,Run,Region,Chain,MID_no,Primer,MID,Patient,Sample_Project,Reads,Species,Sample
0,run169,region1,BCRh,10,NaN,AGTCAGAGGC,LN40,LN-study,2500,human,10-IgA
1,run169,region2,BCRh,10,NaN,AGTCAGAGGC,LN75,LN-study,2500,human,10-IgA
2,run169,region1,BCRh,10,NaN,AGTCATGTCT,LN40,LN-study,2500,human,10-IgD
3,run169,region2,BCRh,10,NaN,AGTCATGTCT,LN75,LN-study,2500,human,10-IgD
4,run169,region1,BCRh,10,NaN,AGTCAGGGGA,LN40,LN-study,2500,human,10-IgG


In [4]:
ig = df["Sample"].tolist()
ig_new = list()
for i in range(len(ig)):
    if "-" in str(ig[i]):
        ig_new.append(ig[i].split("-")[1])
    else:
        ig_new.append('')
df["Sample"] = ig_new

In [5]:
df.head()

,Run,Region,Chain,MID_no,Primer,MID,Patient,Sample_Project,Reads,Species,Sample
0,run169,region1,BCRh,10,NaN,AGTCAGAGGC,LN40,LN-study,2500,human,IgA
1,run169,region2,BCRh,10,NaN,AGTCAGAGGC,LN75,LN-study,2500,human,IgA
2,run169,region1,BCRh,10,NaN,AGTCATGTCT,LN40,LN-study,2500,human,IgD
3,run169,region2,BCRh,10,NaN,AGTCATGTCT,LN75,LN-study,2500,human,IgD
4,run169,region1,BCRh,10,NaN,AGTCAGGGGA,LN40,LN-study,2500,human,IgG


In [6]:
patient = df["Patient"].tolist()
subtype = df["Sample"].tolist()

In [7]:
sample_name = list()
for i in range(len(patient)):
    if subtype[i] != '':
        sample_name.append(patient[i] + "-" + subtype[i])
    else:
        sample_name.append(patient[i])
df["Sample_Name"] = sample_name

In [8]:
len(df)

465

In [9]:
df.to_csv("20190410-Roche-LN-pt_table-reformatted.csv", index=False, sep=";")

## List split files and write linux commands to rename them

In [10]:
import os

In [11]:
mydir = 'split'
myfiles = list()
myrun = list()
myregion = list()
mymid = list()
for f in os.listdir(mydir):
    if f.endswith('.fastq.gz'):
        (run, region, mid) = f.split("-")
        region = region.split("_")[0]
        region = region.replace("r", "region")
        mid = mid.replace(".fastq.gz", "")
        
        myfiles.append(f)
        myrun.append(run)
        myregion.append(region)
        mymid.append(mid)
d = {'Run': myrun, 'Region': myregion, 'MID': mymid, 'File': myfiles}
df_files = pd.DataFrame.from_dict(d)
df_files.head()

,Run,Region,MID,File
0,run162,region2,ACACTGCTCA,run162-r2_L001.assembled-ACACTGCTCA.fastq.gz
1,run162,region2,ACAGGGCTCA,run162-r2_L001.assembled-ACAGGGCTCA.fastq.gz
2,run162,region2,ACATGAGCTC,run162-r2_L001.assembled-ACATGAGCTC.fastq.gz
3,run162,region2,ACTAGGGCTC,run162-r2_L001.assembled-ACTAGGGCTC.fastq.gz
4,run162,region2,AGAACGGCTC,run162-r2_L001.assembled-AGAACGGCTC.fastq.gz


In [15]:
# Combine with the sample information
df_merged = pd.merge(df, df_files, how='left', on=['Run', 'Region', 'MID'])

In [16]:
# Create new file name
sample_name = df_merged['Sample_Name'].tolist()
file_name = df_merged['File'].tolist()
new_name = list()
for i in range(len(sample_name)):
    new_name.append(str(sample_name[i]) + "-" + str(file_name[i]))
df_merged['File_Renamed'] = new_name

In [18]:
df_merged.head()

,Run,Region,Chain,MID_no,Primer,MID,Patient,Sample_Project,Reads,Species,Sample,Sample_Name,File,File_Renamed
0,run169,region1,BCRh,10,NaN,AGTCAGAGGC,LN40,LN-study,2500,human,IgA,LN40-IgA,run169-r1_L001.assembled-AGTCAGAGGC.fastq.gz,LN40-IgA-run169-r1_L001.assembled-AGTCAGAGGC.f...
1,run169,region2,BCRh,10,NaN,AGTCAGAGGC,LN75,LN-study,2500,human,IgA,LN75-IgA,run169-r2_L001.assembled-AGTCAGAGGC.fastq.gz,LN75-IgA-run169-r2_L001.assembled-AGTCAGAGGC.f...
2,run169,region1,BCRh,10,NaN,AGTCATGTCT,LN40,LN-study,2500,human,IgD,LN40-IgD,run169-r1_L001.assembled-AGTCATGTCT.fastq.gz,LN40-IgD-run169-r1_L001.assembled-AGTCATGTCT.f...
3,run169,region2,BCRh,10,NaN,AGTCATGTCT,LN75,LN-study,2500,human,IgD,LN75-IgD,run169-r2_L001.assembled-AGTCATGTCT.fastq.gz,LN75-IgD-run169-r2_L001.assembled-AGTCATGTCT.f...
4,run169,region1,BCRh,10,NaN,AGTCAGGGGA,LN40,LN-study,2500,human,IgG,LN40-IgG,run169-r1_L001.assembled-AGTCAGGGGA.fastq.gz,LN40-IgG-run169-r1_L001.assembled-AGTCAGGGGA.f...


In [23]:
fhOut = open("tmp-rename.sh", "w")
for i in range(len(df_merged)):
    if str(df_merged['File'][i]).endswith(".fastq.gz"):
        print("mv", df_merged['File'][i], df_merged['File_Renamed'][i], file=fhOut)
fhOut.close()
print("Wrote tmp-rename.sh to disk")

Wrote tmp-rename.sh to disk
